<a href="https://colab.research.google.com/github/Narae00/DeepLearning/blob/main/AutoEncoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
#AutoEncoder.py

import tensorflow as tf

class AutoEncoder:
  def __init__(self):
    self.encoder= None
    self.decoder= None
    self.en_decoder= None
    self.relu= tf.keras.activations.relu
    self.tanh= tf.keras.activations.tanh
    self.input_output_dim= 784
    self.encoder_hidden_layers= [200, 200]
    self.decoder_hidden_layers= [200, 200]
    self.code_dim= 32

In [5]:
def build_model(self):
  # Build Encoder
  encoder_input= tf.keras.layers.Input(shape=(self.input_output_dim, ), dtype=tf.float32)
  encoder_h_layer= encoder_input
  for dim in self.encoder_hidden_layers:
    encoder_h_layer= tf.keras.layers.Dense(
        units=dim, activation=self.relu, use_bias=True
        )(encoder_h_layer)
  code = tf.keras.layers.Dense(
      units=self.code_dim, activation=self.tanh, use_bias=True
      )(encoder_h_layer)
  self.encoder= tf.keras.models.Model(inputs=encoder_input, outputs=code)
  # Build Decoder
  decoder_input= tf.keras.layers.Input(shape=(self.code_dim, ), dtype=tf.float32)
  decoder_h_layer= decoder_input
  for dim in self.decoder_hidden_layers:
    decoder_h_layer= tf.keras.layers.Dense(
        units=dim, activation=self.relu, use_bias=True
        )(decoder_h_layer)

  decoder_output= tf.keras.layers.Dense(
    units=self.input_output_dim, activation=None, use_bias=True
  )(decoder_h_layer)
  self.decoder= tf.keras.models.Model(inputs=decoder_input, outputs=decoder_output)

  # En-Decoder
  vae_output= self.decoder(code)
  self.en_decoder= tf.keras.models.Model(inputs=encoder_input, outputs=vae_output)
  optimizer_alg= tf.keras.optimizers.Adam(learning_rate=0.001)
  mse= tf.keras.losses.mse
  self.en_decoder.compile(optimizer=optimizer_alg, loss=mse)

def fit(self, x, y, batch_size, epochs):
  self.en_decoder.fit(x=x, y=y, batch_size=batch_size, epochs=epochs)

def save_weights(self, save_path):
  self.en_decoder.save_weights(save_path)

def load_weights(self, load_path):
  self.en_decoder.load_weights(load_path)

In [6]:
#main.py
from MNISTData import MNISTData
from AutoEncoder import AutoEncoder


if __name__=="__main__":
  print("Hi. I am an Auto Encoder Trainer.")
  batch_size = 32
  num_epochs = 5

  data_loader = MNISTData()
  data_loader.load_data()

  x_train = data_loader.x_train
  input_output_dim = data_loader.in_out_dim
  auto_encoder = AutoEncoder()
  auto_encoder.build_model()
  auto_encoder.fit(x=x_train, y=x_train, batch_size=batch_size, epochs=num_epochs)

  save_path = "./models/ae_model.weights.h5"
  auto_encoder.save_model(save_path)
  print("load model weights from %s" %save_path)


ModuleNotFoundError: No module named 'MNISTData'

In [ ]:
#main_test.py


import tensorflow as tf
from MNISTData import MNISTData
from AutoEncoder import AutoEncoder
import numpyas np

if __name__ == "__main__":
  print("Hi. I am an AutoEncoderTester.")
  batch_size= 32
  num_epochs= 5
  data_loader= MNISTData()
  data_loader.load_data()
  x_train= data_loader.x_train
  input_output_dim= data_loader.in_out_dim

  auto_encoder= AutoEncoder()
  auto_encoder.build_model()
  load_path= "./model/ae_model.weights.h5"
  print("load model weights from %s" % load_path)
  auto_encoder.load_weights(load_path)
  # print for test
  num_test_items= 56
  test_data= data_loader.x_test[0:num_test_items, :]
  test_label= data_loader.y_test[0:num_test_items]
  test_data_x_print= test_data.reshape(num_test_items, data_loader.width, data_loader.height)

  print("const by codes")
  reconst_data= auto_encoder.en_decoder.predict(test_data)
  reconst_data_x_print= reconst_data.reshape(num_test_items, data_loader.width, data_loader.height)
  reconst_data_x_print= tf.math.sigmoid(reconst_data_x_print)
  MNISTData.print_56_pair_images(test_data_x_print, reconst_data_x_print, test_label)
  print("const by code means for each digit")
  avg_codes= np.zeros([10, 32])
  avg_add_cnt= np.zeros([10])

  latent_vecs= auto_encoder.encoder.predict(test_data)
  for i, label in enumerate(test_label):
    avg_codes[label] = latent_vecs[i]
    avg_add_cnt[label] += 1.0
    for i in range(10):
      if avg_add_cnt[label] > 0.1:
        avg_codes[i] /= avg_add_cnt[label]

  avg_code_tensor= tf.convert_to_tensor(avg_codes)
  reconst_data_by_vecs= auto_encoder.decoder.predict(avg_code_tensor)
  reconst_data_x_by_mean_print= reconst_data_by_vecs.reshape(10, data_loader.width, data_loader.height)
  label_list= [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
  MNISTData.print_10_images(reconst_data_x_by_mean_print, label_list)